# Exercise 4: Tool Definition

In this exercise, you'll define a tool and see how the model asks to use it.

By the end, you'll understand:
- How to define a tool by passing a Python function to Gemini
- That the model doesn't **call** tools — it returns `function_call` parts **asking you** to call them
- The structure of a function call response
- How to disable automatic function calling so you can inspect the raw response

## Step 1: Setup

In [ ]:
from google import genai
from google.genai import types

client = genai.Client()

## Step 2: Define a tool

Gemini can infer tool schemas from Python functions. Just define a function with:
- **Type hints** → parameter types
- **Docstring** → tool description

We disable automatic function calling so we can see the raw tool-call response.

In [ ]:
def get_weather(location: str) -> str:
    """Get the current weather in a given location."""
    pass

## Step 3: Send a message that should trigger tool use

When we ask about the weather, the model should recognize that it needs to use the `get_weather` tool.

In [ ]:
response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents="What is the weather in Tokyo?",
    config=types.GenerateContentConfig(
        tools=[get_weather],
        automatic_function_calling=types.AutomaticFunctionCallingConfig(disable=True),
    ),
)

## Step 4: Inspect the response

The response contains `function_call` parts instead of text. The model is **pausing** and asking us to run a tool.

In [ ]:
response.candidates[0].content

Look at each part's `function_call` — it has:
- `name` — which tool the model wants to use
- `args` — the arguments as a **dict** (unlike OpenAI where it's a JSON string you need to parse!)

In [ ]:
for part in response.candidates[0].content.parts:
    if part.function_call:
        fc = part.function_call
        print(f"Tool: {fc.name}")
        print(f"  Arguments: {fc.args}")
        print(f"  Type of args: {type(fc.args)}")

## Key takeaway

The model never actually calls your tool. It tells you **which** tool to call and **what arguments** to pass. Unlike OpenAI, Gemini gives you `args` as a ready-to-use dict — no `json.loads()` needed! It's your job to execute the tool and send the result back. That's what the next exercise covers.